In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("prf.csv")
df

,result,prediction
0,a,a
1,a,d
2,c,b
3,b,b
4,d,e
5,d,d
6,a,a
7,a,e
8,a,b
9,c,a


# Precision,Recall,Accuracy By CM

In [3]:
cm = pd.crosstab(df['result'],df['prediction'], rownames =['result'], colnames = ['prediction'])
cm

prediction,a,b,c,d,e
result,,,,,
a,3,1,0,1,1
b,1,4,0,0,1
c,2,2,3,0,0
d,0,1,0,4,1
e,1,1,0,1,2


In [17]:
def pre(cm):
    res = []
    for i in (np.diag(cm),list(np.sum(cm, axis=1))):
        res.append(np.diag(cm)/list(np.sum(cm, axis=1)))
        return res

In [18]:
def rec(cm):
    res = []
    for i in (np.diag(cm),list(np.sum(cm))):
        res.append(np.diag(cm)/list(np.sum(cm)))
        return res

In [19]:
def f1(cm):
    res = []
    p = pre(cm)
    r = rec(cm)
    for i,j in zip(p,r):
        res.append((2*i*j)/(i+j))
    return res

In [20]:
def accuracy(cm):
    return((np.sum(np.diag(cm)))/(np.sum(np.sum(cm))))

In [21]:
pre(cm)

[array([0.5       , 0.66666667, 0.42857143, 0.66666667, 0.4       ])]

In [22]:
rec(cm)

[array([0.42857143, 0.44444444, 1.        , 0.66666667, 0.4       ])]

In [23]:
f1(cm)

[array([0.46153846, 0.53333333, 0.6       , 0.66666667, 0.4       ])]

In [24]:
accuracy(cm)

0.5333333333333333

# Precision,Recall By Query

In [5]:
#precision 
#out of retrieved results , which fraction is relevant
#setosa
df.query('result == "a" and prediction=="a"').shape[0]/df.query('result == "a" ').shape[0]

0.5

In [6]:
#virginica
df.query('result == "b" and prediction=="b"').shape[0]/df.query('result == "b" ').shape[0]

0.6666666666666666

In [7]:
#versicolor
df.query('result == "c" and prediction=="c"').shape[0]/df.query('result == "c" ').shape[0]

0.42857142857142855

In [8]:
#Recall
#out of all relevant results, which fraction is retrieved
#setosa
df.query('result == "a" and prediction=="a"').shape[0]/df.query('prediction == "a" ').shape[0]

0.42857142857142855

In [34]:
#virginica
df.query('retrieved == "virginica" and gold=="virginica"').shape[0]/df.query('gold == "virginica" ').shape[0]

1.0

In [35]:
#versicolor
df.query('retrieved == "versicolor" and gold=="versicolor"').shape[0]/df.query('gold == "versicolor" ').shape[0]

0.7894736842105263

# MAP, p@k

In [30]:
retrieved = df.loc[:,'result'].to_numpy()
retrieved

array(['a', 'a', 'c', 'b', 'd', 'd', 'a', 'a', 'a', 'c', 'c', 'e', 'e',
       'e', 'e', 'b', 'a', 'b', 'c', 'e', 'c', 'd', 'c', 'c', 'd', 'd',
       'd', 'b', 'b', 'b'], dtype=object)

In [31]:
gold =df.loc[:,'prediction'].to_numpy()
gold

array(['a', 'd', 'b', 'b', 'e', 'd', 'a', 'e', 'b', 'a', 'c', 'e', 'a',
       'b', 'd', 'b', 'a', 'b', 'c', 'e', 'c', 'd', 'a', 'b', 'd', 'b',
       'd', 'b', 'e', 'a'], dtype=object)

In [63]:
def precision(gold,retrieved):
    
   
    p_a = 0
    p_b = 0
    p_c = 0
    p_d = 0
    p_e = 0
    for a,b in zip(gold,retrieved):
        
        if(a==b):
            if(a=='a'):
                p_a+=1
            if(a=='b'):
                p_b+=1
            if(a=='c'):
                p_c+=1
            if(a=='d'):
                p_d+=1
            if(a=='e'):
                p_e+=1
    if(list(retrieved).count('a')==0):
        pr_a = 0
    else:
        pr_a = p_a/list(retrieved).count('a')
    if(list(retrieved).count('b')==0):
        pr_b = 0
    else:
        pr_b = p_b/list(retrieved).count('b')
    if(list(retrieved).count('c')==0):
        pr_c = 0
    else:
        pr_c = p_c/list(retrieved).count('c')
    if(list(retrieved).count('d')==0):
        pr_d = 0
    else:
        pr_d = p_d/list(retrieved).count('d')
    if(list(retrieved).count('e')==0):
        pr_e = 0
    else:
        pr_e = p_e/list(retrieved).count('e')
        
    print("Precision is:")
    print(pr_a)
    print(pr_b)
    print(pr_c)
    print(pr_d)
    print(pr_e)
    

In [64]:
def precision_k(gold,retrieved,k):
    retrieved_k = retrieved[:k]
    gold_k = gold[:k]
    
    return precision(gold_k,retrieved_k)

In [66]:
precision_k(gold,retrieved,10)

Precision is:
0.4
1.0
0.0
0.5
0


In [69]:
#mean average precision
#calculate average precision of each query
#then find mean of all of them

def mean_average_precision(gold,retrieved):
    p_a = []
    p_b = []
    p_c = []
    p_d = []
    p_e = []
    i = 1
    pa = 0
    pb = 0
    pc = 0
    pd = 0
    pe = 0
    
    for a,b in zip(gold,retrieved):
        if(a==b):
            if(a=='a'):
                pa += 1
                p_a.append(pa/i)
            if(a=='b'):
                pb += 1
                p_b.append(pb/i)
            if(a=='c'):
                pc += 1
                p_c.append(pc/i)
            if(a=='d'):
                pd += 1
                p_d.append(pd/i)
            if(a=='e'):
                pe += 1
                p_e.append(pe/i)
        i+=1
            
    avg_p_a = sum(p_a)/len(p_a)
    avg_p_b = sum(p_b)/len(p_b)
    avg_p_c = sum(p_c)/len(p_c)
    avg_p_d = sum(p_d)/len(p_d)
    avg_p_e = sum(p_e)/len(p_e)
 
    
    return((avg_p_a + avg_p_b + avg_p_c + avg_p_d + avg_p_e)/5)
            

In [70]:
mean_average_precision(gold,retrieved)

0.19892667013642246

# Fallout

In [71]:
#fallout
df.query('result=="a" and result != prediction ').shape[0]/df.query('result != prediction').shape[0]

0.21428571428571427